# Basic usage of LRP
This example will show you best practices for using LRP,
building on the basics shown in the *Getting started* section.

We start out by loading a small convolutional neural network:

In [1]:
using ExplainableAI
using Flux

model = Chain(
    Chain(
        Conv((3, 3), 3 => 8, relu; pad=1),
        Conv((3, 3), 8 => 8, relu; pad=1),
        MaxPool((2, 2)),
        Conv((3, 3), 8 => 16; pad=1),
        BatchNorm(16, relu),
        Conv((3, 3), 16 => 8, relu; pad=1),
        BatchNorm(8, relu),
    ),
    Chain(
        Flux.flatten,
        Dense(2048 => 512, relu),
        Dropout(0.5),
        Dense(512 => 100, softmax)
    ),
);

This model contains two chains: the convolutional layers and the fully connected layers.

## Model preparation
### Stripping the output softmax
When using LRP, it is recommended to explain output logits instead of probabilities.
This can be done by stripping the output softmax activation from the model
using the `strip_softmax` function:

In [2]:
model = strip_softmax(model)

Chain(
  Chain(
    Conv((3, 3), 3 => 8, relu, pad=1),  # 224 parameters
    Conv((3, 3), 8 => 8, relu, pad=1),  # 584 parameters
    MaxPool((2, 2)),
    Conv((3, 3), 8 => 16, pad=1),       # 1_168 parameters
    BatchNorm(16, relu),                # 32 parameters, plus 32
    Conv((3, 3), 16 => 8, relu, pad=1),  # 1_160 parameters
    BatchNorm(8, relu),                 # 16 parameters, plus 16
  ),
  Chain(
    Flux.flatten,
    Dense(2048 => 512, relu),           # 1_049_088 parameters
    Dropout(0.5),
    Dense(512 => 100),                  # 51_300 parameters
  ),
)         # Total: 16 trainable arrays, 1_103_572 parameters,
          # plus 4 non-trainable, 48 parameters, summarysize 4.213 MiB.

If you don't remove the output softmax,
model checks will fail.

### Canonizing the model
LRP is not invariant to a model's implementation.
Applying the `GammaRule` to two linear layers in a row will yield different results
than first fusing the two layers into one linear layer and then applying the rule.
This fusing is called "canonization" and can be done using the `canonize` function:

In [3]:
model = canonize(model)

Chain(
  Conv((3, 3), 3 => 8, relu, pad=1),    # 224 parameters
  Conv((3, 3), 8 => 8, relu, pad=1),    # 584 parameters
  MaxPool((2, 2)),
  Conv((3, 3), 8 => 16, relu, pad=1),   # 1_168 parameters
  Conv((3, 3), 16 => 8, relu, pad=1),   # 1_160 parameters
  BatchNorm(8, relu),                   # 16 parameters, plus 16
  Flux.flatten,
  Dense(2048 => 512, relu),             # 1_049_088 parameters
  Dropout(0.5),
  Dense(512 => 100),                    # 51_300 parameters
)         # Total: 14 trainable arrays, 1_103_540 parameters,
          # plus 2 non-trainable, 16 parameters, summarysize 4.212 MiB.

### Flattening the model
ExplainableAI.jl's LRP implementation supports nested Flux Chains and Parallel layers.
However, it is recommended to flatten the model before analyzing it.

LRP is implemented by first running a forward pass through the model,
keeping track of the intermediate activations, followed by a backward pass
that computes the relevances.

To keep the LRP implementation simple and maintainable,
ExplainableAI.jl does not pre-compute "nested" activations.
Instead, for every internal chain, a new forward pass is run to compute activations.

By "flattening" a model, this overhead can be avoided.
For this purpose, ExplainableAI.jl provides the function `flatten_model`:

In [4]:
model_flat = flatten_model(model)

Chain(
  Conv((3, 3), 3 => 8, relu, pad=1),    # 224 parameters
  Conv((3, 3), 8 => 8, relu, pad=1),    # 584 parameters
  MaxPool((2, 2)),
  Conv((3, 3), 8 => 16, relu, pad=1),   # 1_168 parameters
  Conv((3, 3), 16 => 8, relu, pad=1),   # 1_160 parameters
  BatchNorm(8, relu),                   # 16 parameters, plus 16
  Flux.flatten,
  Dense(2048 => 512, relu),             # 1_049_088 parameters
  Dropout(0.5),
  Dense(512 => 100),                    # 51_300 parameters
)         # Total: 14 trainable arrays, 1_103_540 parameters,
          # plus 2 non-trainable, 16 parameters, summarysize 4.212 MiB.

This function is called by default when creating an LRP analyzer.
Note that we pass the unflattened model to the analyzer, but `analyzer.model` is flattened:

In [5]:
analyzer = LRP(model)
analyzer.model

Chain(
  Conv((3, 3), 3 => 8, relu, pad=1),    # 224 parameters
  Conv((3, 3), 8 => 8, relu, pad=1),    # 584 parameters
  MaxPool((2, 2)),
  Conv((3, 3), 8 => 16, relu, pad=1),   # 1_168 parameters
  Conv((3, 3), 16 => 8, relu, pad=1),   # 1_160 parameters
  BatchNorm(8, relu),                   # 16 parameters, plus 16
  Flux.flatten,
  Dense(2048 => 512, relu),             # 1_049_088 parameters
  Dropout(0.5),
  Dense(512 => 100),                    # 51_300 parameters
)         # Total: 14 trainable arrays, 1_103_540 parameters,
          # plus 2 non-trainable, 16 parameters, summarysize 4.212 MiB.

If this flattening is not desired, it can be disabled
by passing the keyword argument `flatten=false` to the `LRP` constructor.

## LRP rules
By default, the `LRP` constructor will assign the `ZeroRule` to all layers.

In [6]:
LRP(model)

LRP(
  Conv((3, 3), 3 => 8, relu, pad=1)  => ZeroRule(),
  Conv((3, 3), 8 => 8, relu, pad=1)  => ZeroRule(),
  MaxPool((2, 2))                    => ZeroRule(),
  Conv((3, 3), 8 => 16, relu, pad=1) => ZeroRule(),
  Conv((3, 3), 16 => 8, relu, pad=1) => ZeroRule(),
  BatchNorm(8, relu)                 => ZeroRule(),
  Flux.flatten                       => ZeroRule(),
  Dense(2048 => 512, relu)           => ZeroRule(),
  Dropout(0.5)                       => ZeroRule(),
  Dense(512 => 100)                  => ZeroRule(),
)

This analyzer will return heatmaps that look identical to `InputTimesGradient`.

LRP's strength lies in assigning different rules to different layers,
based on their functionality in the neural network[^1].
ExplainableAI.jl implements many LRP rules out of the box,
but it is also possible to *implement custom rules*.

To assign different rules to different layers,
use one of the composites presets,
or create your own composite, as described in
*Assigning rules to layers*.

In [7]:
composite = EpsilonPlusFlat() # using composite preset EpsilonPlusFlat

Composite(
  GlobalTypeMap(  # all layers
    Flux.Conv               => ZPlusRule(),
    Flux.ConvTranspose      => ZPlusRule(),
    Flux.CrossCor           => ZPlusRule(),
    Flux.Dense              => EpsilonRule{Float32}(1.0f-6),
    typeof(NNlib.dropout)   => PassRule(),
    Flux.AlphaDropout       => PassRule(),
    Flux.Dropout            => PassRule(),
    Flux.BatchNorm          => PassRule(),
    typeof(Flux.flatten)    => PassRule(),
    typeof(MLUtils.flatten) => PassRule(),
    typeof(identity)        => PassRule(),
 ),
  FirstLayerTypeMap(  # first layer
    Flux.Conv          => FlatRule(),
    Flux.ConvTranspose => FlatRule(),
    Flux.CrossCor      => FlatRule(),
    Flux.Dense         => FlatRule(),
 ),
)

In [8]:
LRP(model, composite)

LRP(
  Conv((3, 3), 3 => 8, relu, pad=1)  => FlatRule(),
  Conv((3, 3), 8 => 8, relu, pad=1)  => ZPlusRule(),
  MaxPool((2, 2))                    => ZeroRule(),
  Conv((3, 3), 8 => 16, relu, pad=1) => ZPlusRule(),
  Conv((3, 3), 16 => 8, relu, pad=1) => ZPlusRule(),
  BatchNorm(8, relu)                 => PassRule(),
  Flux.flatten                       => PassRule(),
  Dense(2048 => 512, relu)           => EpsilonRule{Float32}(1.0f-6),
  Dropout(0.5)                       => PassRule(),
  Dense(512 => 100)                  => EpsilonRule{Float32}(1.0f-6),
)

## Computing layerwise relevances
If you are interested in computing layerwise relevances,
call `analyze` with an LRP analyzer and the keyword argument
`layerwise_relevances=true`.

The layerwise relevances can be accessed in the `extras` field
of the returned `Explanation`:

In [9]:
input = rand(Float32, 32, 32, 3, 1) # dummy input for our convolutional neural network

expl = analyze(input, analyzer; layerwise_relevances=true)
expl.extras.layerwise_relevances

11-element Vector{Array{Float32}}:
 [0.015440371 -0.04796141 … -0.009486459 -0.0059399772; -0.0031672693 -0.006933646 … 0.0028045783 -0.017828126; … ; 0.0028196154 0.068093956 … 0.026031038 0.12635392; -0.012230551 -0.17501104 … 0.06540284 -0.0021100608;;; -0.017633334 0.005505965 … -0.00037752994 0.022605991; 0.018536465 -0.054127432 … 0.011018713 -0.0009427859; … ; 0.010912214 0.05915217 … -0.024697088 -0.059113264; 0.00429308 0.025882076 … 0.014640631 -0.020085879;;; -0.0047129905 -0.005584318 … 0.0020465406 -0.0063166316; -0.003244906 -0.009858072 … 0.0012800801 -0.02658166; … ; -0.023745779 -0.019262312 … -0.008813649 -0.007177769; -0.019258786 0.016711578 … 0.13713284 0.0071985414;;;;]
 [0.018990755 0.0 … -0.0 0.0; -0.0034619716 -0.03547479 … 0.0 0.0; … ; 0.03630635 -0.0 … -0.0 0.0; -0.026927402 -0.0 … -0.0 -0.009744562;;; -0.0 -0.0 … -0.0 -0.0; -0.0 0.0 … 0.0 0.0; … ; -0.0 0.0 … 0.0 -0.0; -0.0 -0.0 … -0.0 0.0;;; 0.0 -0.0 … 0.041003276 0.0; -0.00044708667 -0.0 … -0.028429002 -0.0

Note that the layerwise relevances are only kept for layers in the outermost `Chain` of the model.
When using our unflattened model, we only obtain three layerwise relevances,
one for each chain in the model and the output relevance:

In [10]:
analyzer = LRP(model; flatten=false) # use unflattened model

expl = analyze(input, analyzer; layerwise_relevances=true)
expl.extras.layerwise_relevances

11-element Vector{Array{Float32}}:
 [-0.0072697713 -0.013756088 … -0.012450598 -0.08291615; -0.013432466 0.0019369167 … 0.026284223 0.052460548; … ; 0.09004208 -0.024740001 … -0.014878287 -0.05151539; -0.13646473 -0.22518742 … -0.09717751 0.0006391926;;; -0.004842595 0.008442691 … -0.017975366 0.12468483; -0.0023733715 -0.081422485 … -0.068094045 -0.035478752; … ; 0.011479549 0.10746232 … -0.013396142 0.046710756; -0.053162895 0.085395545 … -0.11661572 0.023325764;;; -0.0034672706 -0.0055207172 … 0.039317146 -0.012910203; 0.0016627561 0.06798119 … -0.0014721396 -0.011407747; … ; -0.06788608 -0.06110469 … 0.025078826 0.0013176994; -0.023915747 0.08526903 … -0.13261557 -0.042681515;;;;]
 [0.025250793 0.0 … -0.0 -0.0; -0.0032661648 -0.011430724 … 0.0 0.0; … ; 0.18071458 -0.0 … 0.0 0.0; -0.0062632114 0.0 … 0.0 0.042009987;;; -0.0 -0.0 … 0.0 -0.0; -0.0 0.0 … 0.0 0.0; … ; -0.0 0.0 … -0.0 0.0; -0.0 -0.0 … -0.0 -0.0;;; 0.0 -0.0 … 0.04846616 -0.0; -0.0019552896 -0.0 … 0.034661848 -0.027691517; 

## Performance tips
### Using LRP without a GPU
Since ExplainableAI.jl's LRP implementation makes use of
[Tullio.jl](https://github.com/mcabbott/Tullio.jl),
analysis can be accelerated by loading either
- a package from the [JuliaGPU](https://juliagpu.org) ecosystem,
  e.g. [CUDA.jl](https://github.com/JuliaGPU/CUDA.jl), if a GPU is available
- [LoopVectorization.jl](https://github.com/JuliaSIMD/LoopVectorization.jl)
  if only a CPU is available.

This only requires loading the LoopVectorization.jl package before ExplainableAI.jl:
```julia
using LoopVectorization
using ExplainableAI
```

[^1]: G. Montavon et al., [Layer-Wise Relevance Propagation: An Overview](https://link.springer.com/chapter/10.1007/978-3-030-28954-6_10)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*